In [10]:
%run 'train.py'

Attempted to log scalar metric Regularization Strength::
1.0
Attempted to log scalar metric Max iterations::
1000
Attempted to log scalar metric accuracy:
0.9090541224076885


In [11]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-121494")
exp = Experiment(workspace=ws, name="quick-starts-ws-121494")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121494
Azure region: southcentralus
Subscription id: ac15aef5-0abe-4be6-a0bd-40abc1594138
Resource group: aml-quickstarts-121494


In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'cpu-cluster'
try:
    cpu_cluster = ComputeTarget(workspace = ws,name = cpu_cluster_name)
    print('found the existing cluster,use it.')
except ComputeTargetException:
    compute_config= AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,cpu_cluster_name,compute_config)
    
cpu_cluster.wait_for_completion(show_output = True)

found the existing cluster,use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [13]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--max_iter':choice(1000,1100,1200),'--C':choice(0.2,0.4,0.6,0.8,1.0)})### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2,slack_factor = 0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './',entry_script = 'train.py',compute_target= cpu_cluster)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,hyperparameter_sampling=ps,policy=policy,primary_metric_name= 'accuracy',
                                    primary_metric_goal =PrimaryMetricGoal.MAXIMIZE,max_total_runs=12,max_concurrent_runs=2)### YOUR CODE HERE ###

In [14]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
runs = exp.submit(config=hyperdrive_config)
RunDetails(runs).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [15]:
import joblib
# Get your best run and save the model from that run.
best_run = runs.get_best_run_by_primary_metric()
joblib.dump(best_run,'model.joblib')
### YOUR CODE HERE ###

TypeError: can't pickle _thread.RLock objects

In [21]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
df=TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [22]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)
x_train,x_test,y_train,y_test = train_test_split(x, y,test_size=0.3,random_state=42)

train_set = pd.concat([x_train,y_train],axis=1)

In [23]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_set,
    label_column_name='y',
    n_cross_validations=3)

In [25]:
# Submit your automl run
auto_run = exp.submit(config=automl_config,show_output = True)
### YOUR CODE HERE ###

Running on local machine
Parent Run ID: AutoML_88621f07-561b-43cb-8ac2-474f997efb09

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: BestRunExplainModel. Best run model explanations completed
****************************************************************************************************


In [27]:
# Retrieve and save your best automl model.
print(auto_run.get_best_child())
print(auto_run.get_output())
### YOUR CODE HERE ###

Run(Experiment: quick-starts-ws-121494,
Id: AutoML_88621f07-561b-43cb-8ac2-474f997efb09_49,
Type: None,
Status: Completed)
(Run(Experiment: quick-starts-ws-121494,
Id: AutoML_88621f07-561b-43cb-8ac2-474f997efb09_49,
Type: None,
Status: Completed), Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               scale_pos_weight=1,
                             